In [12]:
import tensorflow as tf
import pathlib
import random
import math
import numpy as np
from PIL import Image

In [13]:
data_dir = pathlib.Path(r'E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped')
data_dir

WindowsPath('E:/NIIT/Semester 7/Capstone Project II/Project/Dataset/UTKface_Aligned_cropped')

In [14]:
train_raw = []
cv_raw = []
# test_raw = []
for i in range(1, 100):
    image_paths = list(map(lambda x: str(x), data_dir.glob(f'UTKFace/{i}_*.jpg')))
    class_label = (i - 1) // 5
    if image_paths:
        random.shuffle(image_paths)
        train_size = math.floor(len(image_paths) * 0.6)
        cv_size = math.floor(len(image_paths) * 0.2)
        train_raw.extend([(p, class_label) for p in image_paths[:train_size]])
        cv_raw.extend([(p, class_label) for p in image_paths[train_size:train_size + cv_size]])
        #test_paths.extend(image_paths[train_size + cv_size:])
random.shuffle(train_raw)
random.shuffle(cv_raw)
#random.shuffle(test_raw)

In [15]:
train_raw

[('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\26_1_4_20161221193703783.jpg.chip.jpg',
  5),
 ('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\26_0_1_20170117200404157.jpg.chip.jpg',
  5),
 ('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\29_0_3_20170119195114531.jpg.chip.jpg',
  5),
 ('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\18_1_1_20170109214355606.jpg.chip.jpg',
  3),
 ('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\1_1_4_20161221200207440.jpg.chip.jpg',
  0),
 ('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\12_1_3_20161220222343139.jpg.chip.jpg',
  2),
 ('E:\\NIIT\\Semester 7\\Capstone Project II\\Project\\Dataset\\UTKface_Aligned_cropped\\UTKFace\\28_0_3_20170104220258078.jpg.chip.jpg',

In [16]:
train_paths, train_labels = zip(*train_raw)
train_paths = list(train_paths)
train_labels = list(train_labels)

In [17]:
cv_paths, cv_labels = zip(*cv_raw)
cv_paths = list(cv_paths)
cv_labels = list(cv_labels)

In [18]:
train_data = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))

In [19]:
cv_data = tf.data.Dataset.from_tensor_slices((cv_paths, cv_labels))

In [20]:
def read_image(image_path, label):
    image = tf.expand_dims(tf.io.decode_image(tf.io.read_file(image_path), dtype = tf.float32), axis = 0)
    #labels = tf.reshape(tf.one_hot(label, 20, axis = 0), (1, 20))
    labels = [label]
    return image, labels

In [21]:
train_data = train_data.map(read_image)

In [22]:
cv_data = cv_data.map(read_image)

In [23]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(64, 3, activation = 'relu', input_shape = (200, 200, 3)))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(128, 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Conv2D(256, 3, activation = 'relu'))
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(20, activation = 'softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 97, 97, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 46, 46, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 135424)            0

In [25]:
model.compile(optimizer = 'sgd', 
              loss = tf.losses.SparseCategoricalCrossentropy(from_logits = False), 
              metrics = ['accuracy'])

In [ ]:
model.fit(train_data, epochs = 10, validation_data = cv_data, batch_size = 32)

In [ ]:
model.save_weights('/kaggle/working/checkpoint')

In [26]:
model.load_weights('./checkpoint')
loss, acc = model.evaluate(cv_data)
print(f'Loss: {loss}, Accuracy: {acc}')

2325/2325 [==============================] - 263s 112ms/step - loss: 2.5129 - accuracy: 0.2151
Loss: $2.5129165649414062, Accuracy: $0.2150537669658661
